# Configuration

> Define the data classes using Pydantic, making it possible to configure the chat application and do input validation.

In [ ]:
#| default_exp config

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
#| hide
from pydantic import BaseModel, Field
from typing import Optional, List, Tuple, Literal
import os
from pathlib import Path
from dotenv import load_dotenv

In [ ]:
#| export
# Load environment variables from .env file
load_dotenv()

True

## The theme of the Gradio Chat App

The AppTheme data class makes it possible to configure the _colors_ and optionally a _logo_.

The logo's should be stored within the data folder of the project for easy acces. In future iterations the logo's can be retrieved from:
- the web
- a database
- a datafolder

In [ ]:
#| export
class AppTheme(BaseModel):
    """Configuration for the visual theme of the app"""
    primary_color: str = Field(default="#007BFF", description="Primary color for UI elements")
    secondary_color: str = Field(default="#6C757D", description="Secondary color for UI elements")
    background_color: str = Field(default="#FFFFFF", description="Background color")
    text_color: str = Field(default="#212529", description="Main text color")
    logo_path: Optional[Path] = Field(default=None, description="Path to logo image")

## The configuration for the workings of the LLM chatbot

First the configuration for the LLM model to use in the `ModelConfig`.

In [ ]:
#| export
class ModelConfig(BaseModel):
    """Configuration for the LLM model"""
    model_name: str = Field(..., description="Name or path of the model to use")
    provider: str = Field(default="huggingface", description="Model provider (huggingface, openai, etc)")
    api_key_env_var: Optional[str] = Field(default=None, description="Environment variable name for API key")
    api_base_url: Optional[str] = Field(default=None, description="Base URL for API reqeuest")
    max_tokens: int = Field(default=1024, description="Maximum tokens to generate")
    temperature: float = Field(default=0.7, description="Temperature for generation")
    stop_sequences: Optional[List[str]] = Field(default=["\nUser:", "<|endoftext|>"], description="Sequences to stop generation")
    
    @property
    def api_key(self) -> Optional[str]:
        """Get the API key from environment variables if specified"""
        if self.api_key_env_var:
            if os.environ.get(self.api_key_env_var):
                return os.environ.get(self.api_key_env_var)
            raise ValueError(f"The environment variable {self.api_key_env_var} is not found in the .env file.")
        return None

Next the configuration of the Message system

In [ ]:
#| export
class Message(BaseModel):
    """A message in a conversation"""
    role: Literal["system", "user", "assistatn"] = Field(..., description="Role of the message sender")
    content: str = Field(..., description="Content of the message")

Then the configuration for the chat implementation. Making sure the application can handle:
- system prompt
- context if applicable
- a start 'user' prompt if applicable
- user input

The other settings that are available in this class can easily be infered from the description in the `ChatAppConfig` class itself.

In [ ]:
#| export
class ChatAppConfig(BaseModel):
    """Main configuration for a chat application"""
    app_name: str = Field(..., description="Name of the application")
    description: str = Field(default="", description="Description of the application")
    system_prompt: str = Field(..., description="System prompt for the LLM")
    starter_prompt: Optional[str] = Field(default=None, description="Initial prompt to start the conversation")
    context_files: List[Path] = Field(default=[], description="List of markdown files for additional context")
    model: ModelConfig
    theme: AppTheme = Field(default_factory=AppTheme)
    show_system_prompt: bool = Field(default=True, description="Whether to show system prompt in UI")
    show_context: bool = Field(default=True, description="Whether to show context in UI")

An example configuration for a chat application could look like:

In [ ]:
test_config = ChatAppConfig(
    app_name="Test App",
    system_prompt="You are a helpful assistant.",
    model=ModelConfig(
        model_name="gpt-3.5-turbo",
        api_key_env_var="TEST_VALUE",
    )
)

print(test_config.model_dump_json(indent=2))

print(f"API Key available: {'Yes' if test_config.model.api_key else 'No'}")

{
  "app_name": "Test App",
  "description": "",
  "system_prompt": "You are a helpful assistant.",
  "starter_prompt": null,
  "context_files": [],
  "model": {
    "model_name": "gpt-3.5-turbo",
    "provider": "huggingface",
    "api_key_env_var": "TEST_VALUE",
    "api_base_url": null,
    "max_tokens": 1024,
    "temperature": 0.7,
    "stop_sequences": [
      "\nUser:",
      "<|endoftext|>"
    ]
  },
  "theme": {
    "primary_color": "#007BFF",
    "secondary_color": "#6C757D",
    "background_color": "#FFFFFF",
    "text_color": "#212529",
    "logo_path": null
  },
  "show_system_prompt": true,
  "show_context": true
}
API Key available: Yes


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()